In [1]:
# https://support.travelpayouts.com/hc/ru/categories/200358578-API-%D0%B8-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5
# https://support.travelpayouts.com/hc/ru/articles/203956163-Aviasales-API-%D0%B4%D0%BE%D1%81%D1%82%D1%83%D0%BF%D0%B0-%D0%BA-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC-%D0%B4%D0%BB%D1%8F-%D1%83%D1%87%D0%B0%D1%81%D1%82%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2-%D0%BF%D0%B0%D1%80%D1%82%D0%BD%D0%B5%D1%80%D1%81%D0%BA%D0%BE%D0%B9-%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D1%8B#15

In [2]:
# https://api.travelpayouts.com/documentation?_gl=1*bd467j*_ga*MTA5NjE0OTQ5NS4xNjc2OTk5MDUz*_ga_1WLL0NEBEH*MTY3Njk5OTA1My4xLjEuMTY3Njk5OTEwNi43LjAuMA..#grouped-flights-tickets-and-search-by-ticket-39-s-price-v3

In [3]:
ACCESS_TOKEN = 'b8848ff67e92768d2d1977862372d796'

origin = ['LED', 'KZN', 'MOW'] #'KWE'
final_destination = ['BEG'] #['LED', 'MOW', 'TLL', 'HEL'] #'LED' 'KLO'
range_date = ['2023-03-01', '2023-04-30']
#number_place = 0

time_sleep = 1
date_start, date_end = range_date[0], range_date[1]

#url = "https://api.travelpayouts.com/aviasales/v3/grouped_prices" # самый рабочий вариант, но не онятно почему всего один билет
#url = 'https://api.travelpayouts.com/aviasales/v3/prices_for_dates'
#url = "https://api.travelpayouts.com/v2/prices/latest"
#url = "https://api.travelpayouts.com/v1/prices/direct" #departure_date
url = "https://api.travelpayouts.com/v2/prices/month-matrix"
# group = ['DME', 'SVO', 'VKO', 'ZIA']

wishful_point = [] #'KLO'


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service = service)

In [5]:
import requests
import json
import codecs
import os

from itertools import product
from tqdm import tqdm_notebook as tqdm
from collections import Counter

from datetime import datetime, timedelta
from time import sleep
import pandas as pd

add_zero = lambda x: '0' + str(x) if len(str(x)) < 2 else str(x)
# Используйте тут ваш путь к chromedriver!




In [6]:
f = codecs.open('airports.json', encoding='utf-8', mode='r')
data = json.load(f)

In [7]:
cities = list(map(lambda x: x['city_code'], data))
points = list(map(lambda x: x['code'], data))
cities_points = {city:[] for city in cities}
list(map(lambda x, y: cities_points[x].append(y) , cities, points))
cities_points = {k: v for k, v in cities_points.items() if len(v) > 1}
cities_points = list(cities_points.keys())

mapper =  dict(zip(points, cities))
mapper = {k: v for k, v in mapper.items() if v in cities_points}


In [8]:
url_routes = 'http://api.travelpayouts.com/data/routes.json'
routes = json.loads(requests.get(url_routes).text)
route_chain = list(map(lambda x: [x['departure_airport_iata'], x['arrival_airport_iata']], routes))

def group_airport(route, mapper):
    if route[1] in mapper.keys(): route[1] = mapper[route[1]]
    if route[0] in mapper.keys(): route[0] = mapper[route[0]]
    return route



In [9]:
route_chain = list(map(lambda x: group_airport(x, mapper), route_chain))
route_chain = list(set(map(lambda x: tuple(x), route_chain)))

In [10]:
zone = ['Asia/Aden',
 'Asia/Almaty',
 'Asia/Amman',
 'Asia/Anadyr',
 'Asia/Aqtau',
 'Asia/Aqtobe',
 'Asia/Ashgabat',
 'Asia/Baghdad',
 'Asia/Bahrain',
 'Asia/Baku',
 'Asia/Bangkok',
 'Asia/Beirut',
 'Asia/Bishkek',
 'Asia/Brunei',
 'Asia/Chita',
 'Asia/Choibalsan',
 'Asia/Chongqing',
 'Asia/Colombo',
 'Asia/Damascus',
 'Asia/Dhaka',
 'Asia/Dili',
 'Asia/Dubai',
 'Asia/Dushanbe',
 'Asia/Gaza',
 'Asia/Ho_Chi_Minh',
 'Asia/Hong_Kong',
 'Asia/Hovd',
 'Asia/Irkutsk',
 'Asia/Istanbul',
 'Asia/Jakarta',
 'Asia/Jayapura',
 'Asia/Jerusalem',
 'Asia/Kabul',
 'Asia/Kamchatka',
 'Asia/Karachi',
 'Asia/Kathmandu',
 'Asia/Kolkata',
 'Asia/Krasnoyarsk',
 'Asia/Kuala_Lumpur',
 'Asia/Kuching',
 'Asia/Kuwait',
 'Asia/Macau',
 'Asia/Magadan',
 'Asia/Makassar',
 'Asia/Manila',
 'Asia/Muscat',
 'Asia/Nicosia',
 'Asia/Novokuznetsk',
 'Asia/Novosibirsk',
 'Asia/Omsk',
 'Asia/Oral',
 'Asia/Phnom_Penh',
 'Asia/Pontianak',
 'Asia/Pyongyang',
 'Asia/Qatar',
 'Asia/Qyzylorda',
 'Asia/Rangoon',
 'Asia/Riyadh',
 'Asia/Sakhalin',
 'Asia/Samarkand',
 'Asia/Seoul',
 'Asia/Shanghai',
 'Asia/Singapore',
 'Asia/Srednekolymsk',
 'Asia/Taipei',
 'Asia/Tashkent',
 'Asia/Tbilisi',
 'Asia/Tehran',
 'Asia/Thimphu',
 'Asia/Tokyo',
 'Asia/Ulaanbaatar',
 'Asia/Ust-Nera',
 'Asia/Vientiane',
 'Asia/Vladivostok',
 'Asia/Yakutsk',
 'Asia/Yekaterinburg',
 'Asia/Yerevan',
'Indian/Antananarivo',
 'Indian/Christmas',
 'Indian/Cocos',
 'Indian/Comoro',
 'Indian/Mahe',
 'Indian/Maldives',
 'Indian/Mauritius',
 'Indian/Mayotte',
 'Indian/Reunion',
 'Pacific/Apia',
 'Pacific/Auckland',
 'Pacific/Chatham',
 'Pacific/Chuuk',
 'Pacific/Easter',
 'Pacific/Efate',
 'Pacific/Enderbury',
 'Pacific/Fiji',
 'Pacific/Funafuti',
 'Pacific/Galapagos',
 'Pacific/Gambier',
 'Pacific/Guadalcanal',
 'Pacific/Guam',
 'Pacific/Honolulu',
 'Pacific/Johnston',
 'Pacific/Kiritimati',
 'Pacific/Kosrae',
 'Pacific/Kwajalein',
 'Pacific/Majuro',
 'Pacific/Marquesas',
 'Pacific/Midway',
 'Pacific/Nauru',
 'Pacific/Niue',
 'Pacific/Norfolk',
 'Pacific/Noumea',
 'Pacific/Pago_Pago',
 'Pacific/Palau',
 'Pacific/Pohnpei',
 'Pacific/Port_Moresby',
 'Pacific/Rarotonga',
 'Pacific/Saipan',
 'Pacific/Tahiti',
 'Pacific/Tarawa',
 'Pacific/Tongatapu',
 'Pacific/Wake',
 'Pacific/Wallis']

directions = list(filter(lambda x: x['time_zone'] in zone, data))
#directions = list(filter(lambda x: x['country_code'] in ['CN'], directions))
directions = ['BEG'] #list(map(lambda x: x['city_code'], directions)) , 'KAZ', 'MOW'


In [11]:
def get_next_chain(arrivals, route_chain, final_destination, final = False):
    if final:
        next_chain = filter(lambda x: (x[0] in arrivals) & (x[1] in final_destination), route_chain)
    else:
        next_chain = filter(lambda x: x[0] in arrivals, route_chain)
    return next_chain

def get_arrivals(required_routes):
    if type(required_routes[0][0]) == str:
        arrivals = list(map(lambda x: x[1], required_routes))
    else: 
        arrivals = list(set(map(lambda x: x[-1][1], required_routes)))
    return arrivals

def get_product(required_routes, next_chain):
    if type(required_routes[0][0]) == str:
        required_routes = list(filter(lambda x: x[-2][1] == x[-1][0], product(required_routes, next_chain)))
    else: 
        required_routes = list(filter(lambda x: x[-2][-1][1] == x[-1][0], product(required_routes, next_chain)))
    return required_routes

def get_routes(origin, final_destination, route_chain, number_flights = 4):
    if number_flights == 1:
        required_routes = list(set(filter(lambda x: (x[0] in origin) & (x[1] in final_destination), route_chain)))
    else:
        required_routes = list(set(filter(lambda x: x[0] in origin, route_chain)))
        for n in range(1, number_flights):
            arrivals = get_arrivals(required_routes)
            if n + 1 == number_flights:
                next_chain = get_next_chain(arrivals, route_chain, final_destination, True)
            else:    
                next_chain = get_next_chain(arrivals, route_chain, final_destination)
            required_routes = get_product(required_routes, next_chain)           
    return required_routes

def delete_chains(final_destination, rout):
    s = 1
    for chain in rout:
        if chain[-1] not in final_destination:
            s = s + 1
        else:
            break
    rout = rout[:s]
    return rout
            
def flat_route(route):
    root = route
    result = []
    while type(root[-2]) != str:
        result.append(root[-1])
        root = root[-2]
        
    result.append(root)
    result.reverse()
    return result

In [12]:
def get_wishful_routes(route, wishful_point):
    result = False
    if len(wishful_point) != 0:
        for chain in route:
            if chain[1] in wishful_point or chain[0] in wishful_point:
                result = True
        return result
    else: return True
    
    
def clean_routes(route):
    c = dict(Counter([point for chain in route for point in chain]))
    if 3 in list(c.values()): return False
    else: return True

In [13]:
required_routes = get_routes(origin, final_destination, route_chain, number_flights = 1)
required_routes = list(map(flat_route, required_routes))
required_routes = list(map(lambda x: delete_chains(final_destination, x), required_routes))
required_routes = list(filter(lambda x: get_wishful_routes(x, wishful_point), required_routes))
required_routes = list(filter(lambda x: clean_routes(x), required_routes))

In [14]:
#origin = directions
final_destination = directions
required_routes = list(product(origin, final_destination))

required_routes = list(map(lambda x: [x], required_routes))

In [15]:
all_points = list(set([point for route in required_routes for chain in route for point in chain]))

In [16]:
def list_dates(datetime_start, datetime_end):
    delta = timedelta(days = 1)
    range_dates = []
    date = datetime_start
    while date <= datetime_end:
        range_dates.append(date)
        date = date + delta   
    return range_dates

def open_json(path):
    try:
        with open(path, encoding='utf-8', mode = 'r') as file:
            json_base = json.load(file)
    except FileNotFoundError:
        json_base = []
    return json_base

def write_json(path, json_base):
    with open(path, encoding='utf-8', mode = 'w') as file:
        json.dump(json_base, file)  

def write_file(path, json_base, columns): 
    with open(path, encoding='utf-8', mode = 'a') as file:
        for item in json_base:
            line_file = []
            for key in columns:
                line_file.append(str(item[key]))
            file.write(",".join(line_file) + "\n")

In [17]:
def get_destinations(routes, origin):
    destination_list = list(filter(lambda x: x['departure_airport_iata'] == origin, routes))
    destination_list = list(set(map(lambda x: x['arrival_airport_iata'],destination_list)))
    return destination_list

In [18]:
current_time = datetime.utcnow() - timedelta(days = 10)
current_time

datetime.datetime(2023, 2, 11, 20, 1, 42, 918937)

In [19]:
def update_browser_query(path, days = 5):
    browser_query = open_json(path)
    current_time = datetime.utcnow() - timedelta(days = days)
    browser_query = list(filter(lambda x: datetime.strptime(x['timestamp'], 
                                                            '%Y-%m-%d %H:%M:%S') > current_time, browser_query))  
    write_json(path, browser_query)  

def check_browser_query(date, origin, destination): 
    rewrite = True
    browser_query = open_json('browser_query.json')
    checker = list(filter(lambda x: ((x['date'] == date) & (x['origin'] == origin) & 
                                     (x['destination'] == destination)), browser_query))    
    if len(checker) != 0: rewrite = False          
    return rewrite   

def add_browser_query(date, origin, destination): 
    browser_query = open_json('browser_query.json')
    browser_query.append({'date':date, 'origin':origin, 
         'destination':destination, 'timestamp':datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')})
    write_json('browser_query.json', browser_query)


In [20]:
def update_price_base(path, days = 10):
    columns = ['actual', 'depart_date', 'destination', 'distance', 'duration', 'found_at', 'gate', 'number_of_changes',
     'origin', 'return_date', 'show_to_affiliates', 'trip_class', 'value']  
    if os.path.exists(path):
        data = pd.read_csv(path, parse_dates = ['found_at'])
        current_time = datetime.utcnow() - timedelta(days = days)
        data = data[data['found_at'] > current_time]
        data.to_csv(path, index = False)
    else:
        with open(path, encoding='utf-8', mode = 'a') as file:
            file.write(",".join(columns) + "\n")
        data = pd.read_csv(path, parse_dates = ['found_at'])
    return data

def check_ticket_price(date, origin, destination, base):
    rewrite = base[(base['depart_date'] == date) 
                     & (base['origin'] == origin) 
                     & (base['destination'] == destination)].empty      
    return rewrite


def add_ticket_price(ticket, columns):   
    write_file('ticket_base.txt', ticket, columns)
    
def upload_ticket(token, chain, date, url, ticket_base):
    ticket = []
    columns = list(ticket_base.columns)
    origin, destination = chain
    if check_ticket_price(date, origin, destination, ticket_base):
        params = {'currency': 'rub', 'show_to_affiliates':'false', 'token' : token, 'one_way':'true',
                'origin': origin, 'destination': destination, 'month':date, 'limit':1000}
        response = requests.get(url, params=params)
        if 'errors' not in json.loads(response.text)['data']:
            ticket = json.loads(response.text)['data']
            add_ticket_price(ticket, columns)
        else:
            pass


In [21]:
def get_price_routes(datetime_start, datetime_end, required_routes):
    tickets = []
    ticket_base = update_price_base('ticket_base.txt')
    columns = list(ticket_base.columns)
    
    update_browser_query('browser_query.json')
    stack = []
    
    parameters = list(product(required_routes, list_dates(datetime_start, datetime_end)))

    for route, datetime_object in tqdm(parameters, total=len(parameters), desc='send_query'):
        stack.append((route, datetime_object))
        month, day = add_zero(datetime_object.month), add_zero(datetime_object.day)
        date = datetime_object.strftime('%Y-%m-%d')
        for chain in route:
            origin, destination = chain

            url_for_cash = f'https://www.aviasales.ru/search/{origin}{day}{month}{destination}1'
            
            if check_ticket_price(date, origin, destination, ticket_base):
                upload_ticket(ACCESS_TOKEN, chain, date, url, ticket_base)

            if check_browser_query(date, origin, destination) and check_ticket_price(date, origin, destination, ticket_base):
                driver.get(url_for_cash)
                sleep(time_sleep)
                while "Извините, доступ к поиску" in driver.page_source:
                    check_point = datetime.now()
                    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'sleeping 20 minutes')
                    while datetime.now() < check_point + timedelta(minutes = 20):
                        if stack:
                            route2, datetime_object2 = stack.pop(0)
                            date2 = datetime_object2.strftime('%Y-%m-%d')
                            for chain2 in route2:
                                upload_ticket(ACCESS_TOKEN, chain2, date2, url, ticket_base)
                        else: sleep(60)
  
                    driver.get(url_for_cash)

                add_browser_query(date, origin, destination)
                
    
    for route, datetime_object in tqdm(stack, total=len(stack), desc='get_price_from'):
        date = datetime_object.strftime('%Y-%m-%d')
        for chain in route:
            upload_ticket(ACCESS_TOKEN, chain, date, url, ticket_base)

In [22]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km


def distance(rout, data):
    lon1, lat1 = tuple(list(filter(lambda x: (x['city_code'] == rout[0]) 
                               or (x['code'] == rout[0]), data))[0]['coordinates'].values())
    
    lon2, lat2 = tuple(list(filter(lambda x: (x['city_code'] == rout[1]) 
                               or (x['code'] == rout[1]), data))[0]['coordinates'].values())
    
    km = haversine(lon1, lat1, lon2, lat2)
    return km


In [23]:
# distance = list(map(lambda x: sum(map(lambda y: distance(y, data),x)), required_routes))
# required_routes = [x for _,x in sorted(zip(distance,required_routes))]

In [24]:
tickets_list = []

datetime_start = datetime.strptime(date_start, '%Y-%m-%d')
datetime_end = datetime.strptime(date_end, '%Y-%m-%d')

#destination_list = get_destinations(routes, origin)

get_price_routes(datetime_start, datetime_end, required_routes)

# for change in range(number_place):
#     ponts_list =  list(set(map(lambda x: x['destination'], tickets_list[change])))
#     tickets = []
#     for point in tqdm(ponts_list, desc=f'for_{change + 1}_point_destination'):
#         if change + 1 < number_place: destination_list = get_destinations(routes, point) + final_destination
#         else: destination_list = final_destination
#         tickets = tickets + get_price_routes(datetime_start, datetime_end, point, destination_list)    
            
#     tickets_list.append(tickets)  


C:\Users\ilyachulkov\AppData\Local\Temp\ipykernel_12776\3590872952.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for route, datetime_object in tqdm(parameters, total=len(parameters), desc='send_query'):


send_query:   0%|          | 0/183 [00:00<?, ?it/s]

C:\Users\ilyachulkov\AppData\Local\Temp\ipykernel_12776\3590872952.py:42: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for route, datetime_object in tqdm(stack, total=len(stack), desc='get_price_from'):


get_price_from:   0%|          | 0/183 [00:00<?, ?it/s]

In [25]:
# !git add ticket_base.txt
# !git add browser_query.json
# !git commit -m "update base"
# !git push origin master

In [26]:

#url = "https://api.travelpayouts.com/aviasales/v3/grouped_prices" # самый рабочий вариант, но не онятно почему всего один билет
url = 'https://api.travelpayouts.com/aviasales/v3/prices_for_dates'
#url = "https://api.travelpayouts.com/v2/prices/latest"
#url = "https://api.travelpayouts.com/v1/prices/direct" #departure_date
#url = "https://api.travelpayouts.com/v2/prices/month-matrix"

In [48]:
date = '2023-08-23'

params = {'token' : ACCESS_TOKEN, 'one_way':'true',
        'origin': 'LED', 'destination': 'BEG', 'month':date, 'limit':1000}
response = requests.get(url, params=params)

In [49]:
json.loads(response.text)

{'success': True,
 'data': [{'value': 31907,
   'trip_class': 0,
   'show_to_affiliates': True,
   'origin': 'LED',
   'destination': 'BEG',
   'gate': 'Kupi.com',
   'depart_date': '2023-08-30',
   'return_date': '',
   'number_of_changes': 1,
   'found_at': '2023-02-21T19:52:31Z',
   'duration': 2310,
   'distance': 1795,
   'actual': True},
  {'value': 32181,
   'trip_class': 0,
   'show_to_affiliates': True,
   'origin': 'LED',
   'destination': 'BEG',
   'gate': 'Kupi.com',
   'depart_date': '2023-08-27',
   'return_date': '',
   'number_of_changes': 1,
   'found_at': '2023-02-21T19:23:34Z',
   'duration': 3455,
   'distance': 1795,
   'actual': True},
  {'value': 32431,
   'trip_class': 0,
   'show_to_affiliates': True,
   'origin': 'LED',
   'destination': 'BEG',
   'gate': 'Kupi.com',
   'depart_date': '2023-08-01',
   'return_date': '',
   'number_of_changes': 2,
   'found_at': '2023-02-20T09:07:58Z',
   'duration': 1580,
   'distance': 1795,
   'actual': True},
  {'value': 32

In [29]:
from bs4 import BeautifulSoup
import requests
import re

import os
import sys
import configparser
import telebot
import time

import pandas as pd

In [50]:
link = "https://poisk.avianity.ru/flights/BEG1803LED1"

In [51]:
response = requests.get(link)
soup = BeautifulSoup(response.text, 'lxml')


In [52]:
soup

<!DOCTYPE html>
<html>
<head>
<script>
	window.GEOIP = {"country_code":"ru"};
    </script>
<script>  dataLayer = [{ 'pageCategory': 'flights' }]; </script>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push(

        {'gtm.start': new Date().getTime(),event:'gtm.js'}
        );var f=d.getElementsByTagName(s)[0],
        j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
        'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
        })(window,document,'script','dataLayer','GTM-M47KB56');</script>
<!-- End Google Tag Manager -->
<script data-cfasync="false">
          
          window.TPWLCONFIG = {"language":"ru","currency":"rub","app_link":true,"agencies":null,"engine_type":"combined","hotel":null,"affiliate":{"marker":19314},"default_origin":null,"default_destination":null,"host":"poisk.avianity.ru","redirect_to_mobile":false,"show_hotels":true,"active_tab":"avia","https_redirect":true,"market":"RU"